# nbd

> Extra command line utilities for nbdev

In [ ]:
#| default_exp nbd

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.all import *
import itertools as it
import os
from ghapi.all import *

import configparser
from pathlib import Path

from oztools.core import *
from oztools.gh import *

import subprocess

import json

from glob import glob
import re

In [ ]:
#| export
@call_parse
def nbd_new(name:str, description:str, license:str="Apache-2.0", private:bool=False):
    "Create a new nbdev project and setup github repo for it"
    repo, cloned = gh_new_repo_fn(name, description, license, private)
    os.chdir(repo.name)
    subprocess.run(["nbdev_new"])
    subprocess.run(["nbdev_install_hooks"])
    subprocess.run(["nbdev_prepare"])
    subprocess.run(["nbdev_clean"])
    commit_and_push(cloned, "Initial commit")
    setup_pages(repo.name)

In [ ]:
#| export
def new_notebook_template(name, description):
    template = {
        'cells': [
            {'cell_type': 'markdown', 'metadata': {},
             'source': [
                    f'# {name}\n',
                    '\n',
                    f'> {description}'
                ]},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [f'#| default_exp {name}']},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [
                 '#| hide\n',
                 'from nbdev.showdoc import *'
             ]},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [
                 '#| export\n',
                 'from fastcore.all import *'
             ]},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [
                 '#| hide\n',
                 'import nbdev; nbdev.nbdev_export()'
             ]},
      ], 'metadata': { 'kernelspec': { 'display_name': 'python3', 'language': 'python', 'name': 'python3' } }, 'nbformat': 4, 'nbformat_minor': 4
    }
    return template

FIXME: Cell below results in an error for some reason

template = new_notebook_template("foo", "Makes foo using bar")
print('\n---\n\n'.join(L(template['cells']).attrgot('source').map(''.join)))

In [ ]:
m = re.match(r'(\d+).*\.ipynb', '99ab_asd.ipynb')
m.group(1)

'99'

In [ ]:
#| export
def zero_pad(num):
    num = str(num)
    return num if len(num) > 1 else f"0{num}"

In [ ]:
test_eq(zero_pad(9), '09')
test_eq(zero_pad(32), '32')

In [ ]:
#| export
@call_parse
def nbd_add(name:str, description:str,
            at:Optional[int] = None # If specified, insert new notebook at a specific position
           ):
    "Add new notebook to the project"

    prev_nbs = L(glob("[0-9]*?*.ipynb"))
    prev_ids = prev_nbs.map(lambda x: int(re.match(r'(\d+).*\.ipynb', x).group(1)))
    prev_id = max(prev_ids)

    new_id = prev_id + 1
    new_id = zero_pad(new_id)
    template = new_notebook_template(name, description)

    with open(f"{new_id}_{name}.ipynb", 'w') as f:
        json.dump(template, f)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()